# Introduction

------------------------------
## Mission - Réalisez une classification supervisée d'images
------------------------------

Vous continuez votre travail au sein de "Place du marché". Vous avez partagé le travail effectué lors de votre mission précédente avec Lead Data Scientist, Linda. Elle vous invite désormais à aller plus loin dans l’analyse d’images. 

Voici le mail qu’elle vous a envoyé.

*Bonjour,*

*Merci beaucoup pour ton travail ! Voici la suite de ta mission :*

*Pourrais-tu réaliser une **classification supervisée à partir des images** ? Je souhaiterais que tu mettes en place une **data augmentation** afin d’optimiser le modèle.*

*De plus, nous souhaitons élargir notre gamme de produits à l’épicerie fine.*

*Pour cela, pourrais-tu tester la collecte de produits à base de “champagne” via l’[API disponible ici](https://developer.edamam.com/food-database-api) ou via l'API Openfood Facts en pièce jointe (ne nécessitant aucune inscription)?*

*Pourrais-tu ensuite nous proposer un **script** ou **notebook Python** permettant une extraction des 10 premiers produits dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image.*

*Enfin, pourrais-tu formaliser dans un **support de présentation** de 30 slides maximum au format PDF **l’ensemble de ta démarche** ainsi que les **résultats** d’analyse les plus pertinents ?*

*Merci encore, bon courage !*

*Linda*

*PS : En pièce jointe, tu trouveras pour t’aider un **exemple** de mise en œuvre de classification supervisée sur un autre dataset.*

# Import des librairies

In [57]:
import requests
import json
import pandas as pd
from math import *
import time

# Script

In [61]:
# Define the header as requested in the doc
headers = {
    "User-Agent": "OC-DS-P6-AD - Linux - Version 1.0",
}
# Define request parameters
params = {
    "action": "process",
    "tagtype_0": "categories",
    "tag_contains_0": "contains",
    "tag_0": "champagne",
    "json": "true",
}
# Create a variable to have the status of the full data acquiring
data_acquiring = True

while data_acquiring:
    # Send the GET request to the search part of the API
    response = requests.get("https://fr.openfoodfacts.org/cgi/search.pl",
                           params=params,
                           headers=headers)
    # Handle the response
    if response.status_code == 200:
        # Get the text version of the response
        content = response.text
        # Load the response into a python dictionary
        content_dict = json.loads(content)
        # Get the total number of products
        number_of_products = content_dict["count"]
        # Get the size of the page 
        page_size = content_dict["page_size"]
        # Number of page
        number_of_pages = ceil(content_dict["count"] / content_dict["page_size"])
        # Convert the dictionary to a dataframe
        products = pd.DataFrame.from_dict(content_dict["products"])
        # Add a sleep to slow down requests
        time.sleep(10)
        # Explore other pages
        for i in range(2, number_of_pages+1):
            # Define request parameters
            new_params = {
                "action": "process",
                "tagtype_0": "categories",
                "tag_contains_0": "contains",
                "tag_0": "champagne",
                "json": "true",
                "page": i,
            }
            # Send the GET request to the search part of the API
            new_response = requests.get("https://fr.openfoodfacts.org/cgi/search.pl",
                                   params=params,
                                   headers=headers)
            # Get the text version of the response
            new_content = new_response.text
            # Load the response into a python dictionary
            content_dict = json.loads(new_content)
            # Convert the dictionary to a dataframe
            new_products = pd.DataFrame.from_dict(content_dict["products"])
            # Concatenate the dataframes
            products = pd.concat([products, new_products])     
            # Add a sleep to slow down requests
            time.sleep(10)  
        data_acquiring = False
    else:
        print(f"Request failed with status code : {response.status_code}") 
        # Add a big sleep to avoid being IP blocked
        time.sleep(60)  

In [62]:
products.head()

,_id,_keywords,added_countries_tags,additives_n,additives_original_tags,additives_tags,allergens,allergens_from_ingredients,allergens_from_user,allergens_hierarchy,...,warning_fr,warning_fr_imported,ingredients_text_debug_tags,generic_name_nl,ingredients_text_nl,ingredients_text_with_allergens_nl,origin_nl,packaging_text_nl,product_name_nl,weighters_tags
0,3113934004147,"[alcoolisee, boisson, canard, champagne, duche...",[],0.0,[],[],,,(nl),[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3258431220000,"[boisson, alcoolisee, effervescent, brut, cham...",[],0.0,[],[],,,(fr),[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3185370729960,"[alcoolisée, boisson, brut, champagne, chandon...",[],NaN,NaN,NaN,,,(fr),[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3049610004104,"[alcoolisee, boisson, brut, champagne, champag...",[],0.0,[],[],,,(fr),[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3282946015837,"[brut, point, sulfite, feuillatte, de, france,...",[],0.0,[],[],en:sulphur-dioxide-and-sulphites,sulfites,(fr) en:sulphur-dioxide-and-sulphites,[en:sulphur-dioxide-and-sulphites],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Sélection des variables à conserver

In [64]:
for variable in products.columns.to_list():
    print(variable)

_id
_keywords
added_countries_tags
additives_n
additives_original_tags
additives_tags
allergens
allergens_from_ingredients
allergens_from_user
allergens_hierarchy
allergens_lc
allergens_tags
amino_acids_prev_tags
amino_acids_tags
brands
brands_tags
categories
categories_hierarchy
categories_lc
categories_old
categories_properties
categories_properties_tags
categories_tags
category_properties
checkers_tags
ciqual_food_name_tags
cities_tags
code
codes_tags
compared_to_category
complete
completeness
correctors_tags
countries
countries_hierarchy
countries_lc
countries_tags
created_t
creator
data_quality_bugs_tags
data_quality_errors_tags
data_quality_info_tags
data_quality_tags
data_quality_warnings_tags
data_sources
data_sources_tags
debug_param_sorted_langs
ecoscore_data
ecoscore_grade
ecoscore_score
ecoscore_tags
editors_tags
emb_codes
emb_codes_tags
entry_dates_tags
expiration_date
food_groups
food_groups_tags
generic_name
generic_name_fr
grades
id
image_front_small_url
image_front_thu

In [71]:
products_simplified = products[["id", 
                                "product_name", 
                                "categories", 
                                "ingredients_text", 
                                "image_url",
                               ]]
products_simplified

,id,product_name,categories,ingredients_text,image_url
0,3113934004147,Canard Duchêne,"Boissons, Boissons alcoolisées, Vins, Vins eff...",Pinots et de Chardonnay,https://images.openfoodfacts.org/images/produc...
1,3258431220000,,"Boissons, Boissons alcoolisées, Vins, Vins eff...",Champagne,https://images.openfoodfacts.org/images/produc...
2,3185370729960,Champagne Impérial Brut,"Boissons et préparations de boissons,Boissons,...",,https://images.openfoodfacts.org/images/produc...
3,3049610004104,Veuve Clicquot Champagne Ponsardin Brut,"Boissons et préparations de boissons, Boissons...",Champagne,https://images.openfoodfacts.org/images/produc...
4,3282946015837,Nicolas Feuillatte,"Boissons, Boissons alcoolisées, Vins, Vins fra...","Champagne, Contient des _sulfites_",https://images.openfoodfacts.org/images/produc...
...,...,...,...,...,...
45,3507400004012,Champagne brut - grande tradition,"Boissons, Boissons alcoolisées, Vins, Vins eff...",,https://images.openfoodfacts.org/images/produc...
46,26026237,Champagne,"Boissons, Boissons alcoolisées, Vins, Vins eff...",NaN,https://images.openfoodfacts.org/images/produc...
47,3049610004203,Veuve Clicquot Champagne Ponsardin Brut,"Getränke,Alkoholische Getränke,Weine,Schaumwei...",,https://images.openfoodfacts.org/images/produc...
48,3043700103814,G.H. MUMM CHAMPAGNE Cordon Rouge,"Boissons, Boissons alcoolisées, Vins, Vins fra...",,https://images.openfoodfacts.org/images/produc...


# Selection des produits et export

In [73]:
products_simplified_short = products_simplified.iloc[:10,]
products_simplified_short

,id,product_name,categories,ingredients_text,image_url
0,3113934004147,Canard Duchêne,"Boissons, Boissons alcoolisées, Vins, Vins eff...",Pinots et de Chardonnay,https://images.openfoodfacts.org/images/produc...
1,3258431220000,,"Boissons, Boissons alcoolisées, Vins, Vins eff...",Champagne,https://images.openfoodfacts.org/images/produc...
2,3185370729960,Champagne Impérial Brut,"Boissons et préparations de boissons,Boissons,...",,https://images.openfoodfacts.org/images/produc...
3,3049610004104,Veuve Clicquot Champagne Ponsardin Brut,"Boissons et préparations de boissons, Boissons...",Champagne,https://images.openfoodfacts.org/images/produc...
4,3282946015837,Nicolas Feuillatte,"Boissons, Boissons alcoolisées, Vins, Vins fra...","Champagne, Contient des _sulfites_",https://images.openfoodfacts.org/images/produc...
5,20712907,Champagne,"Boissons, Boissons alcoolisées, Vins, Vins eff...",NaN,https://images.openfoodfacts.org/images/produc...
6,3113910312013,Champagne Alfred Rothschild et Cie brut,"Boissons, Boissons alcoolisées, Vins, Vins fra...",Champagne brut (contient _sulfites_),https://images.openfoodfacts.org/images/produc...
7,3416181017169,"Champagne AOP, brut","Boissons, Boissons alcoolisées, Vins, Vins eff...",Champagne,https://images.openfoodfacts.org/images/produc...
8,3760017736329,Champagne Paul Menand,"Boissons, Boissons alcoolisées, Vins, Vins eff...",NaN,https://images.openfoodfacts.org/images/produc...
9,3185370283905,Champagne Ruinart,"Boissons, Boissons alcoolisées, Vins, Vins eff...",champagne,https://images.openfoodfacts.org/images/produc...


In [74]:
products_simplified_short.to_csv("top_10_products_infos.csv")